In [10]:
from models import MLDataset
from utils.paths import Paths
from facades import EvaluationFacade
from datetime import datetime

lasso_dataset = MLDataset(f'{Paths.ML_DATASETS_FOLDER}/48sectors_LASSO_learned_in_250125')
lgbm_dataset = MLDataset(f'{Paths.ML_DATASETS_FOLDER}/48sectors_LGBM_learned_in_250125')
en_dataset = MLDataset(f'{Paths.ML_DATASETS_FOLDER}/48sectors_ensembled_learned_in_250125')

In [11]:
ml_dataset = lasso_dataset
ml_dataset =MLDataset(f'{Paths.ML_DATASETS_FOLDER}/LGBM_New48sectors_Ensembled_to2021')
ef = EvaluationFacade(ml_dataset, 
                 start_day = datetime(2022, 1, 1), 
                 end_day = datetime(2025, 12, 31),
                 bin_num=5)
ef.display()

c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axe

読み込みが完了しました。
全期間
母集団の推定（99.7%）
平均リターン: 0.1233% (worst: 0.0661%, best: 0.1805%)
標準偏差: 0.005254 (worst:0.005688, best:0.004877)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.23463 (worst:0.116177, best:0.370046)
最大ドローダウン: 5.0386% (worst:11.0155%, best:2.9656%)
最大ドローダウン日数: 163日 (worst:667日, best:66日)
----------------------------------------------------------------------------------
直近1年
母集団の推定（99.7%）
平均リターン: 0.1038% (worst: 0.0119%, best: 0.1957%)
標準偏差: 0.004868 (worst:0.0056, best:0.004293)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.213269 (worst:0.021277, best:0.455898)
最大ドローダウン: 5.1355% (worst:59.2143%, best:2.1187%)
最大ドローダウン日数: 198日 (worst:19880日, best:43日)
----------------------------------------------------------------------------------
直近3ヶ月
母集団の推定（99.7%）
平均リターン: 0.1439% (worst: -0.0181%, best: 0.3059%)
標準偏差: 0.004162 (worst:0.005614, best:0.003273)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.345763 (worst:-0.032234, best:0.93478

Output()

In [14]:
df = ef.visualizer.metrics_dfs['日次成績'][['LS']].rename(columns={'LS': 'Target'})
df.to_csv('result_現行.csv')